In [1]:
# import argparse
# import importlib
import sys
# import argparse
import numpy as np
import pandas as pd
import datetime as dt
# from dateutil.parser import parse
from Kernel import Kernel
from util import util
from util.order import LimitOrder
from util.oracle.SparseMeanRevertingOracle import SparseMeanRevertingOracle

from agent.ExchangeAgent import ExchangeAgent
from agent.NoiseAgent import NoiseAgent
from agent.ValueAgent import ValueAgent
from agent.market_makers.AdaptiveMarketMakerAgent import AdaptiveMarketMakerAgent
from agent.examples.MomentumAgent import MomentumAgent
from agent.examples.ExampleExperimentalAgent import ExampleExperimentalAgentTemplate, ExampleExperimentalAgent
from model.LatencyModel import LatencyModel

In [6]:
#market config
Exchange_Agent = 1
POV_Market_Maker_Agent = 1
Value_Agents = 100
Momentum_Agents = 25
Noise_Agents = 5000

#agent config
ticker = 'ABM'
seed = 413
log_dir = f'log/experimental_agent_demo_short_2min_long_5min_{seed}_{ticker}'
historical_date = 20200603
start_time = '09:30:00'
end_time = '10:30:00'
verbose = False
fund_vol = None
experimental_agent = True
ea_short_window = '2min'
ea_long_window = '5min'

if __name__ == '__main__':

    # Print system banner.
    system_name = "ABIDES: Agent-Based Interactive Discrete Event Simulation"

    print ("=" * len(system_name))
    print (system_name)
    print ("=" * len(system_name))
    print ()

#     rgs, remaining_args = parser.parse_known_args()

    # if config_help:
    #     parser.print_help()
    #     sys.exit()

#     log_dir = log_dir  # Requested log directory.
#     seed = seed  # Random seed specification on the command line.
    if not seed: seed = int(pd.Timestamp.now().timestamp() * 1000000) % (2 ** 32 - 1)
    np.random.seed(seed)

    util.silent_mode = not verbose
    LimitOrder.silent_mode = not verbose

    exchange_log_orders = True
    log_orders = None
    book_freq = 0

    simulation_start_time = dt.datetime.now()
    print("Simulation Start Time: {}".format(simulation_start_time))
    print("Configuration seed: {}\n".format(seed))
    ########################################################################################################################
    ############################################### AGENTS CONFIG ##########################################################

    # Historical date to simulate.
    historical_date = pd.to_datetime(historical_date)
    mkt_open = historical_date + pd.to_timedelta(start_time)
    mkt_close = historical_date + pd.to_timedelta(end_time)
    agent_count, agents, agent_types = 0, [], []

    # Hyperparameters
    symbol = ticker
    starting_cash = 10000000  # Cash in this simulator is always in CENTS.

    r_bar = 1e5
    sigma_n = r_bar / 10
    kappa = 1.67e-15
    lambda_a = 7e-11

    # Oracle
    symbols = {symbol: {'r_bar': r_bar,
                        'kappa': 1.67e-16,
                        'sigma_s': 0,
                        'fund_vol': fund_vol,
                        'megashock_lambda_a': 2.77778e-18,
                        'megashock_mean': 1e3,
                        'megashock_var': 5e4,
                        'random_state': np.random.RandomState(seed=np.random.randint(low=0, high=2 ** 32, dtype='uint64'))}}

    oracle = SparseMeanRevertingOracle(mkt_open, mkt_close, symbols)

    # 1) Exchange Agent

    #  How many orders in the past to store for transacted volume computation
    # stream_history_length = int(pd.to_timedelta(mm_wake_up_freq).total_seconds() * 100)
    stream_history_length = 25000

    agents.extend([ExchangeAgent(id=0,
                                 name="EXCHANGE_AGENT",
                                 type="ExchangeAgent",
                                 mkt_open=mkt_open,
                                 mkt_close=mkt_close,
                                 symbols=[symbol],
                                 log_orders=exchange_log_orders,
                                 pipeline_delay=0,
                                 computation_delay=0,
                                 stream_history=stream_history_length,
                                 book_freq=book_freq,
                                 wide_book=True,
                                 random_state=np.random.RandomState(seed=np.random.randint(low=0, high=2 ** 32, dtype='uint64')))])
    agent_types.extend("ExchangeAgent")
    agent_count += 1

    # 2) Noise Agents
    num_noise = 5000
    noise_mkt_open = historical_date + pd.to_timedelta("09:00:00")  # These times needed for distribution of arrival times
                                                                    # of Noise Agents
    noise_mkt_close = historical_date + pd.to_timedelta("16:00:00")
    agents.extend([NoiseAgent(id=j,
                              name="NoiseAgent {}".format(j),
                              type="NoiseAgent",
                              symbol=symbol,
                              starting_cash=starting_cash,
                              wakeup_time=util.get_wake_time(noise_mkt_open, noise_mkt_close),
                              log_orders=log_orders,
                              random_state=np.random.RandomState(seed=np.random.randint(low=0, high=2 ** 32, dtype='uint64')))
                   for j in range(agent_count, agent_count + num_noise)])
    agent_count += num_noise
    agent_types.extend(['NoiseAgent'])

    # 3) Value Agents
    num_value = 100
    agents.extend([ValueAgent(id=j,
                              name="Value Agent {}".format(j),
                              type="ValueAgent",
                              symbol=symbol,
                              starting_cash=starting_cash,
                              sigma_n=sigma_n,
                              r_bar=r_bar,
                              kappa=kappa,
                              lambda_a=lambda_a,
                              log_orders=log_orders,
                              random_state=np.random.RandomState(seed=np.random.randint(low=0, high=2 ** 32, dtype='uint64')))
                   for j in range(agent_count, agent_count + num_value)])
    agent_count += num_value
    agent_types.extend(['ValueAgent'])

    # 4) Market Maker Agents

    """
    window_size ==  Spread of market maker (in ticks) around the mid price
    pov == Percentage of transacted volume seen in previous `mm_wake_up_freq` that
           the market maker places at each level
    num_ticks == Number of levels to place orders in around the spread
    wake_up_freq == How often the market maker wakes up

    """

    # each elem of mm_params is tuple (window_size, pov, num_ticks, wake_up_freq, min_order_size)
    mm_params = [('adaptive', 0.025, 10, '10S', 1),
                 ('adaptive', 0.025, 10, '10S', 1)
                 ]

    num_mm_agents = len(mm_params)
    mm_cancel_limit_delay = 50  # 50 nanoseconds

    agents.extend([AdaptiveMarketMakerAgent(id=j,
                                    name="ADAPTIVE_POV_MARKET_MAKER_AGENT_{}".format(j),
                                    type='AdaptivePOVMarketMakerAgent',
                                    symbol=symbol,
                                    starting_cash=starting_cash,
                                    pov=mm_params[idx][1],
                                    min_order_size=mm_params[idx][4],
                                    window_size=mm_params[idx][0],
                                    num_ticks=mm_params[idx][2],
                                    wake_up_freq=mm_params[idx][3],
                                    cancel_limit_delay=mm_cancel_limit_delay,
                                    skew_beta=0,
                                    level_spacing=5,
                                    spread_alpha=0.75,
                                    backstop_quantity=50000,
                                    log_orders=log_orders,
                                    random_state=np.random.RandomState(seed=np.random.randint(low=0, high=2 ** 32,
                                                                                              dtype='uint64')))
                   for idx, j in enumerate(range(agent_count, agent_count + num_mm_agents))])
    agent_count += num_mm_agents
    agent_types.extend('POVMarketMakerAgent')


    # 5) Momentum Agents
    num_momentum_agents = 25

    agents.extend([MomentumAgent(id=j,
                                 name="MOMENTUM_AGENT_{}".format(j),
                                 type="MomentumAgent",
                                 symbol=symbol,
                                 starting_cash=starting_cash,
                                 min_size=1,
                                 max_size=10,
                                 wake_up_freq='20s',
                                 log_orders=log_orders,
                                 random_state=np.random.RandomState(seed=np.random.randint(low=0, high=2 ** 32,
                                                                                           dtype='uint64')))
                   for j in range(agent_count, agent_count + num_momentum_agents)])
    agent_count += num_momentum_agents
    agent_types.extend("MomentumAgent")

    # 6) Experimental Agent

    #### Example Experimental Agent parameters

    if experimental_agent:
        experimental_agent = ExampleExperimentalAgent(
            id=agent_count,
            name='EXAMPLE_EXPERIMENTAL_AGENT',
            type='ExampleExperimentalAgent',
            symbol=symbol,
            starting_cash=starting_cash,
            levels=5,
            subscription_freq=1e9,
            wake_freq='10s',
            order_size=100,
            short_window=ea_short_window,
            long_window=ea_long_window,
            log_orders=True,
            random_state=np.random.RandomState(seed=np.random.randint(low=0, high=2 ** 32, dtype='uint64'))
        )
#     else:
#         experimental_agent = ExampleExperimentalAgentTemplate(
#             id=agent_count,
#             name='EXAMPLE_EXPERIMENTAL_AGENT',
#             type='ExampleExperimentalAgent',
#             symbol=symbol,
#             starting_cash=starting_cash,
#             levels=5,
#             subscription_freq=1e9,args, remaining_args = parser.parse_known_args()

    # if config_help:
    #     parser.print_help()
    #     sys.exit()


    experimental_agents = [experimental_agent]
    agents.extend(experimental_agents)
    agent_types.extend("ExperimentalAgent")
    agent_count += 1


    ########################################################################################################################
    ########################################### KERNEL AND OTHER CONFIG ####################################################

    kernel = Kernel("RMSC03 Kernel", random_state=np.random.RandomState(seed=np.random.randint(low=0, high=2 ** 32,
                                                                                                      dtype='uint64')))

    kernelStartTime = historical_date
    kernelStopTime = mkt_close + pd.to_timedelta('00:01:00')

    defaultComputationDelay = 50  # 50 nanoseconds

    # LATENCY

    latency_rstate = np.random.RandomState(seed=np.random.randint(low=0, high=2**32))
    pairwise = (agent_count, agent_count)

    # All agents sit on line from Seattle to NYC
    nyc_to_seattle_meters = 3866660
    pairwise_distances = util.generate_uniform_random_pairwise_dist_on_line(0.0, nyc_to_seattle_meters, agent_count,
                                                                            random_state=latency_rstate)
    pairwise_latencies = util.meters_to_light_ns(pairwise_distances)

    model_args = {
        'connected': True,
        'min_latency': pairwise_latencies
    }

    latency_model = LatencyModel(latency_model='deterministic',
                                 random_state=latency_rstate,
                                 kwargs=model_args
                                 )
    # KERNEL

    kernel.runner(agents=agents,
                  startTime=kernelStartTime,
                  stopTime=kernelStopTime,
                  agentLatencyModel=latency_model,
                  defaultComputationDelay=defaultComputationDelay,
                  oracle=oracle,
                  log_dir=log_dir)

    simulation_end_time = dt.datetime.now()
    print("Simulation End Time: {}".format(simulation_end_time))
    print("Time taken to run simulation: {}".format(simulation_end_time - simulation_start_time))

ABIDES: Agent-Based Interactive Discrete Event Simulation

Simulation Start Time: 2022-02-16 14:52:02.467564
Configuration seed: 413


--- Simulation time: 1970-01-01 00:00:00.020200603, messages processed: 0, wallclock elapsed: 0 days 00:00:00.000025 ---



TypeError: unsupported operand type(s) for /: 'NoneType' and 'float'

In [9]:
import os
import pandas as pd
import sys

# Auto-detect terminal width.
pd.options.display.width = None
pd.options.display.max_rows = 500000
pd.options.display.max_colwidth = 200




# read_agent_logs.py takes a log directory, reads all agent log files, and produces a summary of
# desired totals or statistics by strategy (type + parameter settings).


# If more than one directory is given, the program aggregates across all of them.

log_dirs = log_dir
stats = []

dir_count = 0
file_count = 0

for log_dir in log_dirs:
  if dir_count % 100 == 0: print ("Completed {} directories".format(dir_count))
  dir_count += 1
  for file in os.listdir(log_dir):
    try:
      df = pd.read_pickle(os.path.join(log_dir,file), compression='bz2')
      # print(df)
      events = [ 'AGENT_TYPE', 'STARTING_CASH', 'ENDING_CASH', 'FINAL_CASH_POSITION', 'MARKED_TO_MARKET' ]
      event = "|".join(events)
      df = df[df['EventType'].str.contains(event)]

      at = df.loc[df['EventType'] == 'AGENT_TYPE', 'Event'][0]
      if 'Exchange' in at:
        # There may be different fields to look at later on.
        continue

      file_count += 1

      sc = df.loc[df['EventType'] == 'STARTING_CASH', 'Event'][0]
      ec = df.loc[df['EventType'] == 'ENDING_CASH', 'Event'][0]
      fcp = df.loc[df['EventType'] == 'FINAL_CASH_POSITION', 'Event'][0]
      fv = df.loc[df['EventType'] == 'MARKED_TO_MARKET', 'Event'][0]

      ret = fcp - sc
      surp = fv - sc
      stats.append({ 'AgentType' : at, 'Return' : ret, 'Surplus' : surp })
    except (IndexError, KeyError):
      continue

df_stats = pd.DataFrame(stats)

print (df_stats.groupby('AgentType').mean())

print ("\nRead {} files in {} log directories.".format(file_count, dir_count))


Completed 0 directories


FileNotFoundError: [Errno 2] No such file or directory: 'e'